In [ ]:
from utils.bigquery_tools import buscar_coches_bq 
import pandas as pd
import traceback
import logging

## paso a paso


In [ ]:

filtros_test = {
    "transmision_preferida": "ambos", # No filtra
    "estetica_min": 5.0,
    "premium_min": 5.0,
    "singular_min": 5.0,
    "tipo_mecanica": ['GASOLINA', 'DIESEL'],
    "tipo_carroceria": ['TODOTERRENO', 'PICKUP', 'SUV', 'CROSSOVER'],
    "plazas_min": 2,
    "penalizar_puertas_bajas": False, # Para este escenario
    "modo_adquisicion_recomendado": "Contado", # O directamente poner precio_maximo
    "precio_max_contado_recomendado": 20000.0,
}

pesos_test = { # Ejemplo de pesos normalizados (asegúrate que los tuyos sumen ~1)
    "estetica": 0.027, 
     "premium": 0.135, 
     "singular": 0.135,
     "altura_libre_suelo": 0.216, 
     "batalla": 0.0, # Batalla sin peso en este ejemplo
     "indice_altura_interior": 0.0, # Índice sin peso en este ejemplo
     "ancho": 0.0, # Ancho sin peso en este ejemplo
     "traccion": 0.270, 
     "reductoras": 0.216
}
k_test = 7

In [ ]:
print("\n\n--- INICIANDO PRUEBA ESCENARIO 1: AVENTURERO EXTREMO (CONTADO) ---")
resultados_s1 = buscar_coches_bq(filtros=filtros_test, pesos=pesos_test, k=k_test)
print(f"Resultados encontrados: {len(resultados_s1)}")
print("Resultados:")
print(resultados_s1)

In [ ]:
print("\n\n--- INICIANDO PRUEBA ESCENARIO 1 ---")
print(f"Filtros: {filtros_test}")
print(f"Pesos: {pesos_test}")

resultados_confort = buscar_coches_bq(
    filtros=filtros_test, 
    pesos=pesos_test, 
    k=k_test
)

print(f"\n--- RESULTADOS ESCENARIO CONFORT ({len(resultados_confort)} coches) ---")
if resultados_confort:
    df_confort = pd.DataFrame(resultados_confort)
    # Definir columnas_a_mostrar con las columnas que deseas mostrar
    columnas_a_mostrar = ['nombre', 'precio_compra_contado', 'tipo_carroceria', 'tipo_mecanica', 'premium', 'singular', 'estetica', 'plazas', 'puertas', 'ancho', 'altura_libre_suelo', 'batalla', 'indice_altura_interior', 'traccion', 'reductoras', 'score_total']
    columnas_existentes_confort = [col for col in columnas_a_mostrar if col in df_confort.columns]
    if columnas_existentes_confort:
        print(df_confort[columnas_existentes_confort].to_markdown(index=False))
    else:
        print("Resultados crudos:", resultados_confort)
else:
    print("No se encontraron resultados.")

In [ ]:
paso_paso_resultados_1 = buscar_coches_bq(filtros=filtros_test, pesos=pesos_test, k=k_test)
print(f"\n--- RESULTADOS ESCENARIO 1 ({len(paso_paso_resultados_1)} coches) ---")
print(paso_paso_resultados_1)

## Prueba 2

In [ ]:
# --- Escenario "Profesional Confort Carretera" (Modo 1 Financiado) ---

filtros_test_confort = {
    # --- De Preferencias y Filtros Técnicos ---
    "transmision_preferida": "automático",
    "estetica_min": 5.0,  # valora_estetica='sí' (post-proc con apasionado='no')
    "premium_min": 1.0,   # apasionado_motor='no'
    "singular_min": 1.0,  # apasionado_motor='no'
    "tipo_mecanica": ["DIESEL", "PHEVD", "PHEVG", "GASOLINA"], # Eficientes para carretera
    "tipo_carroceria": ['3VOL', 'SUV', 'PICKUP'], # RAG para confort, viajes, profesional

    # --- De Lógica de Pasajeros ---
    "plazas_min": 2, # (X=0 + Z=1 + Conductor=1)
    "penalizar_puertas_bajas": False, # Frecuencia ocasional, X=0

    # --- De Lógica Económica (Modo 1 Financiado) ---
    "modo_adquisicion_recomendado": "Financiado",
    "precio_max_contado_recomendado": None,
    "cuota_max_calculada": 458.33, 
    
    # Campos de Modo 2 no aplican aquí
    "modo": None, 
    "submodo": None,
    "pago_contado": None,
    "cuota_max": None,
}

# Pesos simulados:
# Perfil: Estética=SÍ (crudo 5), Premium=NO (1), Singular=NO (1)
# Aventura NINGUNA (altura_libre=0, traccion=0, reductoras=0)
# Usuario NO Alto (batalla=0.5, indice_altura=0.5)
# NO Prioriza Ancho (ancho=0.5)
# Suma crudos: 5+1+1+0+0+0+0.5+0.5+0.5 = 8.5
pesos_test_confort = { 
    "estetica":             5.0 / 8.5,  # ~0.588 (Principal factor diferenciador)
    "premium":              1.0 / 8.5,  # ~0.118
    "singular":             1.0 / 8.5,  # ~0.118
    "altura_libre_suelo":   0.0 / 8.5,  # 0.0
    "traccion":             0.0 / 8.5,  # 0.0
    "reductoras":           0.0 / 8.5,  # 0.0
    "batalla":              0.5 / 8.5,  # ~0.059 (Bajo peso, confort en carretera podría valorarlo un poco)
    "indice_altura_interior": 0.5 / 8.5,  # ~0.059 (Bajo peso)
    "ancho":                0.5 / 8.5,  # ~0.059 (Bajo peso)
}
# print(f"Suma pesos confort: {sum(pesos_test_confort.values())}")

k_test_confort = 5 



In [ ]:

print("\n\n--- INICIANDO PRUEBA ESCENARIO: PROFESIONAL CONFORT CARRETERA ---")
print(f"Filtros: {filtros_test_confort}")
print(f"Pesos: {pesos_test_confort}")

resultados_confort = buscar_coches_bq(
    filtros=filtros_test_confort, 
    pesos=pesos_test_confort, 
    k=k_test_confort
)

print(f"\n--- RESULTADOS ESCENARIO CONFORT ({len(resultados_confort)} coches) ---")
if resultados_confort:
    df_confort = pd.DataFrame(resultados_confort)
    # Definir columnas_a_mostrar con las columnas que deseas mostrar
    columnas_a_mostrar = ['nombre', 'precio_compra_contado', 'tipo_carroceria', 'tipo_mecanica', 'premium', 'singular', 'estetica', 'plazas', 'puertas', 'ancho', 'altura_libre_suelo', 'batalla', 'indice_altura_interior', 'traccion', 'reductoras', 'score_total']
    columnas_existentes_confort = [col for col in columnas_a_mostrar if col in df_confort.columns]
    if columnas_existentes_confort:
        print(df_confort[columnas_existentes_confort].to_markdown(index=False))
    else:
        print("Resultados crudos:", resultados_confort)
else:
    print("No se encontraron resultados.")

In [ ]:
# --- Escenario "Familia Numerosa Eléctrica" (Modo 1 Financiado) ---

filtros_test_familia_electrica = {
    # --- De Preferencias y Filtros Técnicos ---
    "transmision_preferida": "automático", # Eléctrico => automático
    "estetica_min": 5.0,  # valora_estetica='sí'
    "premium_min": 1.0,   # apasionado_motor='no'
    "singular_min": 1.0,  # apasionado_motor='no'
    "tipo_mecanica": ["BEV"], # solo_electricos='sí'
    "tipo_carroceria": ['MONOVOLUMEN', 'FURGONETA', 'SUV'], # RAG para 8 plazas, eléctrico, familia

    # --- De Lógica de Pasajeros ---
    "plazas_min": 8, # ¡Requisito clave!
    "penalizar_puertas_bajas": True, # Frecuente con 2 niños en silla

    # --- De Lógica Económica (Modo 1 Financiado) ---
    "modo_adquisicion_recomendado": "Financiado",
    "precio_max_contado_recomendado": None,
    "cuota_max_calculada": 625.00, 
    
    # Campos de Modo 2 no aplican
    "modo": None, "submodo": None, "pago_contado": None, "cuota_max": None,
}

# Pesos simulados:
# Perfil: Estética=SÍ (crudo 5), Premium=NO (1), Singular=NO (1)
# Aventura OCASIONAL (altura_libre=6, traccion=4, reductoras=1)
# Usuario NO Alto (batalla=0.5, indice_altura=0.5)
# SÍ Prioriza Ancho (ancho=6.0, porque Z=5 >= 2)
# Suma crudos: 5+1+1+6+4+1+0.5+0.5+6.0 = 25.0
pesos_test_familia_electrica = { 
    "estetica":             5.0 / 25.0,  # 0.20
    "premium":              1.0 / 25.0,  # 0.04
    "singular":             1.0 / 25.0,  # 0.04
    "altura_libre_suelo":   6.0 / 25.0,  # 0.24 (Aventura 'ocasional')
    "traccion":             4.0 / 25.0,  # 0.16 (Aventura 'ocasional')
    "reductoras":           1.0 / 25.0,  # 0.04 (Aventura 'ocasional')
    "batalla":              0.5 / 25.0,  # 0.02 (No alto)
    "indice_altura_interior": 0.5 / 25.0,  # 0.02 (No alto)
    "ancho":                6.0 / 25.0,  # 0.24 (Prioriza Ancho)
}
# print(f"Suma pesos familia eléctrica: {sum(pesos_test_familia_electrica.values())}")

k_test_familia_electrica = 5 # Pedir pocos, los coches de 8 plazas eléctricos pueden ser raros

# --- Código para ejecutar en tu notebook ---
# import pandas as pd
# from utils.bigquery_tools import buscar_coches_bq # Ajusta la ruta

# print("\n\n--- INICIANDO PRUEBA ESCENARIO: FAMILIA NUMEROSA ELÉCTRICA ---")
# print(f"Filtros: {filtros_test_familia_electrica}")
# print(f"Pesos: {pesos_test_familia_electrica}")

# resultados_familia_electrica = buscar_coches_bq(
#     filtros=filtros_test_familia_electrica, 
#     pesos=pesos_test_familia_electrica, 
#     k=k_test_familia_electrica
# )

# print(f"\n--- RESULTADOS ESCENARIO FAMILIA ELÉCTRICA ({len(resultados_familia_electrica)} coches) ---")
# if resultados_familia_electrica:
#     df_familia_electrica = pd.DataFrame(resultados_familia_electrica)
#     # Reutilizar columnas_a_mostrar y columnas_existentes de tu prueba anterior
#     # Asegúrate de que 'plazas' y 'puertas' estén en columnas_a_mostrar
#     columnas_a_mostrar = ['nombre', 'marca', 'tipo_mecanica', 'cambio_automatico', 
#                           'traccion', 'reductoras', 'puertas', 'plazas', 'ancho', 
#                           'altura_libre_suelo', 'batalla', 'indice_altura_interior', 
#                           'estetica', 'premium', 'singular', 
#                           'precio_compra_contado', 'score_total']
#     columnas_existentes_fe = [col for col in columnas_a_mostrar if col in df_familia_electrica.columns]
#     if columnas_existentes_fe:
#         print(df_familia_electrica[columnas_existentes_fe].to_markdown(index=False))
#     else:
#         print("Resultados crudos:", resultados_familia_electrica)
# else:
#     print("No se encontraron resultados.")

In [ ]:

print(f"Filtros: {filtros_test_confort}")
print(f"Pesos: {pesos_test_confort}")

resultados_confort = buscar_coches_bq(
    filtros=filtros_test_familia_electrica, 
    pesos=pesos_test_familia_electrica, 
    k=k_test_familia_electrica
)

print(f"\n--- RESULTADOS ESCENARIO CONFORT ({len(resultados_confort)} coches) ---")
if resultados_confort:
    df_confort = pd.DataFrame(resultados_confort)
    # Definir columnas_a_mostrar con las columnas que deseas mostrar
    columnas_a_mostrar = ['nombre', 'precio_compra_contado', 'tipo_carroceria', 'tipo_mecanica', 'premium', 'singular', 'estetica', 'plazas', 'puertas', 'ancho', 'altura_libre_suelo', 'batalla', 'indice_altura_interior', 'traccion', 'reductoras', 'score_total']
    columnas_existentes_confort = [col for col in columnas_a_mostrar if col in df_confort.columns]
    if columnas_existentes_confort:
        print(df_confort[columnas_existentes_confort].to_markdown(index=False))
    else:
        print("Resultados crudos:", resultados_confort)
else:
    print("No se encontraron resultados.")

## Escenario con fallo 1


In [ ]:
from utils.bigquery_tools import buscar_coches_bq 
import pandas as pd
import traceback
import logging

# --- Escenario Derivado de tus Parámetros BQ ---

filtros_test_desde_params_bq = {
    # --- De Preferencias y Filtros Técnicos (simulando salida del agente) ---
    "transmision_preferida": "ambos", # Para no filtrar por transmisión
    "estetica_min": 6.0,
    "premium_min": 6.0,
    "singular_min": 6.0,
    "tipo_mecanica": ['BEV', 'GASOLINA', 'HEVD', 'HEVG', 'PHEVD', 'PHEVG'],
    "tipo_carroceria": ['TODOTERRENO', 'MONOVOLUMEN', 'SUV', 'DESCAPOTABLE'],
    
    # --- De Lógica de Pasajeros ---
    "plazas_min": 7,
    "penalizar_puertas_bajas": False, # Coincide con tu @penalizar_puertas = False

    # --- De Lógica Económica (Simulando Modo 1 Contado o Modo 2 Contado) ---
    # Asumamos que el agente recomendó Modo 1 Contado
    "modo_adquisicion_recomendado": "Contado", 
    "precio_max_contado_recomendado": 25000.0, 
    "cuota_max_calculada": None,
    
    # Campos de Modo 2 directo (para este caso, serían None o no relevantes si se usa Modo 1 Rec.)
    "modo": None, 
    "submodo": None,
    "pago_contado": None, # Si fuera Modo 2, este sería 25000 y modo_adquisicion_recomendado sería None
    "cuota_max": None,
}

# Estos pesos son los que me compartiste, ya normalizados
pesos_test_desde_params_bq = { 
    "estetica": 0.13043478260869565,
    "premium":  0.13043478260869565,
    "singular": 0.13043478260869565,
    "altura_libre_suelo": 0.13043478260869565, # Corresponde a @peso_altura
    "batalla":            0.13043478260869565,
    "indice_altura_interior": 0.10869565217391304,
    "ancho":              0.13043478260869565, 
    "traccion":           0.08695652173913043, 
    "reductoras":         0.021739130434782608
}

k_test_desde_params_bq = 5

In [ ]:

print(f"Filtros: {filtros_test_desde_params_bq}")
print(f"Pesos: {pesos_test_desde_params_bq}")

resultados_confort = buscar_coches_bq(
    filtros=filtros_test_desde_params_bq, 
    pesos=pesos_test_desde_params_bq, 
    k=k_test_desde_params_bq
)

print(f"\n--- RESULTADOS ESCENARIO CONFORT ({len(resultados_confort)} coches) ---")
if resultados_confort:
    df_confort = pd.DataFrame(resultados_confort)
    # Definir columnas_a_mostrar con las columnas que deseas mostrar
    columnas_a_mostrar = ['nombre', 'precio_compra_contado', 'tipo_carroceria', 'tipo_mecanica', 'premium', 'singular', 'estetica', 'plazas', 'puertas', 'ancho', 'altura_libre_suelo', 'batalla', 'indice_altura_interior', 'traccion', 'reductoras', 'score_total']
    columnas_existentes_confort = [col for col in columnas_a_mostrar if col in df_confort.columns]
    if columnas_existentes_confort:
        print(df_confort[columnas_existentes_confort].to_markdown(index=False))
    else:
        print("Resultados crudos:", resultados_confort)
else:
    print("No se encontraron resultados.")

## BQ CLIMA PRUEBAS

In [1]:
# utils/bq_data_lookups.py

import logging
import traceback
from typing import Optional, Dict, Any
from google.cloud import bigquery

# Ajusta la ruta de importación según tu estructura
from graph.perfil.state import InfoClimaUsuario # Necesitamos el modelo Pydantic

# Configura tu proyecto, dataset y tabla de BigQuery
PROJECT_ID = "thecarmentor-mvp2" # Reemplaza si es diferente
DATASET_ID = "web_cars"          # O el dataset donde está tu tabla
TABLE_ZONAS_CLIMAS_ID = "zonas_climas" # El nombre de tu tabla de zonas climáticas

TABLE_FULL_ID_ZONAS = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ZONAS_CLIMAS_ID}"

# Nombres de las columnas en tu tabla BQ que contienen los CPs para cada zona
COLUMNAS_ZONA = [
    "MUNICIPIO_ZBE",
    "ZONA_LLUVIAS",
    "ZONA_NIEBLAS",
    "ZONA_NIEVE",
    "ZONA_CLIMA_MONTA",
    "ZONA_GLP",
    "ZONA_GNV"
]

def obtener_datos_climaticos_por_cp(codigo_postal_str: str) -> Optional[InfoClimaUsuario]:
    """
    Consulta la tabla de zonas climáticas en BigQuery para un código postal dado.

    Args:
        codigo_postal_str: El código postal del usuario (como string).

    Returns:
        Un objeto InfoClimaUsuario con los flags booleanos correspondientes,
        o None si ocurre un error o el CP no produce datos.
    """
    logging.info(f"Buscando información climática para el CP: {codigo_postal_str} en {TABLE_FULL_ID_ZONAS}")
    
    if not codigo_postal_str or not codigo_postal_str.isdigit() or len(codigo_postal_str) != 5:
        logging.warning(f"CP inválido proporcionado para búsqueda climática: {codigo_postal_str}")
        return None

    try:
        client = bigquery.Client(project=PROJECT_ID)
        
        # Intentar convertir el CP a INTEGER para la query, ya que tus columnas BQ son INTEGER
        try:
            cp_int = int(codigo_postal_str)
        except ValueError:
            logging.warning(f"CP '{codigo_postal_str}' no es un entero válido para la query BQ.")
            return InfoClimaUsuario(codigo_postal_consultado=codigo_postal_str, cp_valido_encontrado=False)

        # Construir las partes del SELECT para la query dinámicamente
        select_parts = []
        for col_zona in COLUMNAS_ZONA:
            # Cada parte del SELECT verifica si el CP existe en la columna de zona correspondiente
            # y devuelve TRUE o FALSE para esa zona.
            select_parts.append(
                f"EXISTS(SELECT 1 FROM `{TABLE_FULL_ID_ZONAS}` WHERE {col_zona} = @cp_param) AS {col_zona}"
            )
        
        select_clause = ",\n       ".join(select_parts)
        
        sql = f"""
        SELECT
            {select_clause}
        """
        
        params = [
            bigquery.ScalarQueryParameter("cp_param", "INT64", cp_int),
        ]

        logging.debug(f"Ejecutando query de clima BQ: {sql} con params: {cp_int}")
        print(f"DEBUG (BQ Clima) ► Query: {sql}") # Para depuración
        print(f"DEBUG (BQ Clima) ► Params: {cp_int}") # Para depuración

        job_config = bigquery.QueryJobConfig(query_parameters=params)
        query_job = client.query(sql, job_config=job_config)
        
        results = list(query_job.result()) # Espera y obtiene los resultados

        if results and len(results) == 1:
            row = results[0] # Debería devolver una sola fila con todos los booleanos
            logging.info(f"Datos climáticos encontrados para CP {codigo_postal_str}: {dict(row)}")
            
            # Crear y devolver el objeto InfoClimaUsuario
            # Los nombres de campo en InfoClimaUsuario deben coincidir con COLUMNAS_ZONA
            clima_data = {col: bool(getattr(row, col, False)) for col in COLUMNAS_ZONA}
            clima_data["cp_valido_encontrado"] = True # Se procesó, aunque todos pudieran ser False
            clima_data["codigo_postal_consultado"] = codigo_postal_str
            return InfoClimaUsuario(**clima_data)
        else:
            logging.warning(f"No se encontró una fila de resultados única para el CP {codigo_postal_str} o la query devolvió múltiples filas.")
            # Devolver un objeto con defaults (todos False) pero indicando que el CP se consultó
            return InfoClimaUsuario(codigo_postal_consultado=codigo_postal_str, cp_valido_encontrado=False)

    except Exception as e:
        logging.error(f"Error al obtener datos climáticos para CP {codigo_postal_str} de BigQuery: {e}")
        traceback.print_exc()
        # En caso de error, devolver un objeto indicando que no se pudo procesar
        return InfoClimaUsuario(codigo_postal_consultado=codigo_postal_str, cp_valido_encontrado=False)




In [3]:
# --- Ejemplo de uso (para probar en un notebook o localmente si tienes credenciales) ---
if __name__ == "__main__":
    # Configurar logging para ver los mensajes de la función
    logging.basicConfig(level=logging.INFO)
    test_cp = "46009" # Un CP de ejemplo
    clima_info = obtener_datos_climaticos_por_cp(test_cp)
    if clima_info:
        print(f"\nInformación climática para {test_cp}:")
        print(clima_info.model_dump_json(indent=2))
    else:
        print(f"No se pudo obtener información para {test_cp}")

    # test_cp_no_existente = "00000"
    # clima_info_no = obtener_datos_climaticos_por_cp(test_cp_no_existente)
    # if clima_info_no:
    #     print(f"\nInformación climática para {test_cp_no_existente}:")
    #     print(clima_info_no.model_dump_json(indent=2))
    # else:
    #     print(f"No se pudo obtener información para {test_cp_no_existente}")

INFO:root:Buscando información climática para el CP: 46009 en thecarmentor-mvp2.web_cars.zonas_climas


DEBUG (BQ Clima) ► Query: 
        SELECT
            EXISTS(SELECT 1 FROM `thecarmentor-mvp2.web_cars.zonas_climas` WHERE MUNICIPIO_ZBE = @cp_param) AS MUNICIPIO_ZBE,
       EXISTS(SELECT 1 FROM `thecarmentor-mvp2.web_cars.zonas_climas` WHERE ZONA_LLUVIAS = @cp_param) AS ZONA_LLUVIAS,
       EXISTS(SELECT 1 FROM `thecarmentor-mvp2.web_cars.zonas_climas` WHERE ZONA_NIEBLAS = @cp_param) AS ZONA_NIEBLAS,
       EXISTS(SELECT 1 FROM `thecarmentor-mvp2.web_cars.zonas_climas` WHERE ZONA_NIEVE = @cp_param) AS ZONA_NIEVE,
       EXISTS(SELECT 1 FROM `thecarmentor-mvp2.web_cars.zonas_climas` WHERE ZONA_CLIMA_MONTA = @cp_param) AS ZONA_CLIMA_MONTA,
       EXISTS(SELECT 1 FROM `thecarmentor-mvp2.web_cars.zonas_climas` WHERE ZONA_GLP = @cp_param) AS ZONA_GLP,
       EXISTS(SELECT 1 FROM `thecarmentor-mvp2.web_cars.zonas_climas` WHERE ZONA_GNV = @cp_param) AS ZONA_GNV
        
DEBUG (BQ Clima) ► Params: 46009


INFO:root:Datos climáticos encontrados para CP 46009: {'MUNICIPIO_ZBE': True, 'ZONA_LLUVIAS': False, 'ZONA_NIEBLAS': False, 'ZONA_NIEVE': False, 'ZONA_CLIMA_MONTA': False, 'ZONA_GLP': True, 'ZONA_GNV': True}



Información climática para 46009:
{
  "MUNICIPIO_ZBE": true,
  "ZONA_LLUVIAS": false,
  "ZONA_NIEBLAS": false,
  "ZONA_NIEVE": false,
  "ZONA_CLIMA_MONTA": false,
  "ZONA_GLP": true,
  "ZONA_GNV": true,
  "cp_valido_encontrado": true,
  "codigo_postal_consultado": "46009"
}
